In [1]:
import matplotlib
import numpy as np
import os
import torch
import cv2

/home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image

In [3]:
%cd SuperGluePretrainedNetwork/

/home/ichuviliaeva/ocr_course/course_ocr/task1/SuperGluePretrainedNetwork


In [4]:
from models.superpoint import SuperPoint
from models.utils import read_image

In [5]:
from models.superglue import SuperGlue

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
gpu = torch.device('cuda:0')

In [7]:
DATASET_PATH = Path('/DATA/ichuviliaeva/midv500_compressed/')
assert DATASET_PATH.exists(), DATASET_PATH.absolute()
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

gt_root =  /DATA/ichuviliaeva/midv500_compressed/12_deu_drvlic_new/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/12_deu_drvlic_new/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/33_mac_id/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/33_mac_id/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/50_xpo_id/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/50_xpo_id/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/38_rou_drvlic/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/38_rou_drvlic/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/27_hrv_passport/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/27_hrv_passport/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/44_ukr_id/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/44_ukr_id/images
gt_root =  /DATA/ichuviliaeva/midv500_compressed/23_fin_drvlic/ground_truth
img_root =  /DATA/ichuviliaeva/midv500_compressed/23_f

(50, course_ocr_t1.data.MidvPackage)

In [40]:
spoint = SuperPoint({'keypoint_threshold': 0.05}).to(gpu)
sglue = SuperGlue(config={'match_threshold': 0.5}).to(gpu)

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


In [51]:
class baseline_glu:
    def __init__(self, template_item):
        self.template_item = template_item
        
        self.template_img, self.template_img_t, _ = read_image(
            self.template_item.img_path,
            device=gpu,
            resize=(int(self.template_item.image_size[0]), int(self.template_item.image_size[1])), 
            rotation=0, 
            resize_float=True
        )
        
        self.template_kps = spoint({'image': self.template_img_t})     
        
    def run_superglue(self, item_kps, item_img_t):
        sg_input = dict()
        
        for (k, v) in self.template_kps.items():
            sg_input[k + '0'] = torch.stack(v, axis=0).to(gpu)
        sg_input['image0'] = self.template_img_t
        
        for (k, v) in item_kps.items():
            sg_input[k + '1'] = torch.stack(v, axis=0).to(gpu)
        sg_input['image1'] = item_img_t
        
        with torch.no_grad():
            sg_output = sglue(sg_input)
            
        idx0 = sg_output['matches0'] > -1
        template_kps_cleared = sg_input['keypoints0'][0][idx0[0]]
        idx1 = sg_output['matches0'][idx0]
        item_kps_cleared = sg_input['keypoints1'][0][idx1]
        return template_kps_cleared, item_kps_cleared
    
    def compute_crop(self, H):
        w, h = self.template_item.image_size
        h = float(h)
        w = float(w)
        corners = [[0, 0], [w, 0], [w, h], [0, h]]
        corners_cropped = []
        
        for c in corners:
            vec = H @ np.array([c[0], c[1], 1])
            corners_cropped.append([vec[0] / vec[-1], vec[1] / vec[-1]])
        return np.array(corners_cropped) / np.array([w, h])
    
    def predict(self, item):
        item_img, item_img_t, _ = read_image(
            item.img_path,
            device=gpu,
            resize=(int(item.image_size[0]), int(item.image_size[1])), 
            rotation=0, 
            resize_float=True
        )
        
        item_kps = spoint({'image': item_img_t})
        template_kps_cleared, item_kps_cleared = self.run_superglue(item_kps, item_img_t)
        (H, mask) = cv2.findHomography(np.array(template_kps_cleared.detach().cpu()),
                                       np.array(item_kps_cleared.detach().cpu()), 
                                       method=cv2.RANSAC)
        return self.compute_crop(H)

In [52]:
glu = baseline_glu(data_packs[1].template_item)

In [53]:
prediction = glu.predict(data_packs[1][0])

In [54]:
data_packs[1][0].show(prediction)
plt.show()
plt.savefig("../try_res0.png")
plt.close()

In [55]:
print(prediction)

[[ 0.00906106  0.31367037]
 [ 0.18580544  0.24764679]
 [ 0.25025022  0.09129412]
 [-0.06204612 -0.16309195]]


In [56]:
for i in range(10):
    prediction = glu.predict(data_packs[1][i + 15])
    data_packs[1][i].show(prediction)
    plt.show()
    plt.savefig("../try_res" + str(i) + ".png")
    plt.close()